# Bayes ingenuo

Este programa clasifica correos electrónicos como spam o ham utilizando el alrogítmo de bayes ingenuo.

In [ ]:
installed.packages()[,1]

abind                ade4         AmesHousing             anytime 
            "abind"              "ade4"       "AmesHousing"           "anytime" 
             arules           arulesViz             askpass          assertthat 
           "arules"         "arulesViz"           "askpass"        "assertthat" 
          backports           base64enc                  BH                 bit 
        "backports"         "base64enc"                "BH"               "bit" 
              bit64              bitops                blob                brew 
            "bit64"            "bitops"              "blob"              "brew" 
               brio               broom       broom.helpers               bslib 
             "brio"             "broom"     "broom.helpers"             "bslib" 
                 ca              cachem               callr                 car 
               "ca"            "cachem"             "callr"               "car" 
            carData               caret             caTools          cellranger 
          "carData"             "caret"           "caTools"        "cellranger" 
                cli               clipr               clock          clusterSim 
              "cli"             "clipr"             "clock"        "clusterSim" 
        collections          colorspace          commonmark              config 
      "collections"        "colorspace"        "commonmark"            "config" 
           corrplot             cowplot               cpp11              crayon 
         "corrplot"           "cowplot"             "cpp11"            "crayon" 
        credentials           crosstalk                crul                curl 
      "credentials"         "crosstalk"              "crul"              "curl" 
            cvTools           cyclocomp          data.table                 DBI 
          "cvTools"         "cyclocomp"        "data.table"               "DBI" 
             dbplyr          dendextend            DEoptimR                desc 
           "dbplyr"        "dendextend"          "DEoptimR"              "desc" 
           devtools             diagram         DiceKriging             diffobj 
         "devtools"           "diagram"       "DiceKriging"           "diffobj" 
             digest             diptest             downlit               dplyr 
           "digest"           "diptest"           "downlit"             "dplyr" 
             dslabs                  DT              dtplyr               e1071 
           "dslabs"                "DT"            "dtplyr"             "e1071" 
            ellipse            ellipsis             emmeans        estimability 
          "ellipse"          "ellipsis"           "emmeans"      "estimability" 
           evaluate           extrafont         extrafontdb          factoextra 
         "evaluate"         "extrafont"       "extrafontdb"        "factoextra" 
         FactoMineR               fansi              farver             fastmap 
       "FactoMineR"             "fansi"            "farver"           "fastmap" 
             fdm2id          flashClust           flexclust             flexmix 
           "fdm2id"        "flashClust"         "flexclust"           "flexmix" 
              float         fontawesome   fontBitstreamVera      fontLiberation 
            "float"       "fontawesome" "fontBitstreamVera"    "fontLiberation" 
         fontquiver             forcats             foreach            forecast 
       "fontquiver"           "forcats"           "foreach"          "forecast" 
              forge                 fpc            fracdiff                  fs 
            "forge"               "fpc"          "fracdiff"                "fs" 
              furrr              future        future.apply              gargle 
            "furrr"            "future"      "future.apply"            "gargle" 
              gclus             gdtools            generics                gert 
            "gclus"           "gdtools"      

In [ ]:
usePackage <- function(p) 
{
  if (!is.element(p, installed.packages()[,1]))
    install.packages(p, repos = "https://cran.itam.mx/")
  suppressPackageStartupMessages(require(p, character.only = TRUE, quietly  = TRUE))
}

In [ ]:
usePackage('R.utils')
usePackage('tm')

## Descarga los datos 

In [ ]:
download.mails <- function(url, dir_name, file_name){

  if (!file.exists(dir_name)) {
    dir.create(dir_name)  
  }
  
  download.file(url, destfile=file.path(dir_name, paste0(file_name,".tar.bz2")) )
  bunzip2(file.path(dir_name, paste0(file_name,".tar.bz2")))
  
  untar(file.path(dir_name, paste0(file_name,".tar")), exdir = dir_name)
  
  if (file.exists(file.path(dir_name, paste0(file_name,".tar")))) {
    file.remove(file.path(dir_name, paste0(file_name,".tar")))
  }
  
}

In [ ]:
dir_name <- "data"
file_name <- "easy_ham_2"
url <- "http://spamassassin.apache.org/old/publiccorpus/20030228_easy_ham_2.tar.bz2"

download.mails(url, dir_name, file_name)

[1] TRUE

In [ ]:
url <- "http://spamassassin.apache.org/old/publiccorpus/20030228_hard_ham.tar.bz2"
file_name <- "hard_ham"

download.mails(url, dir_name, file_name)

[1] TRUE

In [ ]:
url <- "http://spamassassin.apache.org/old/publiccorpus/20030228_spam_2.tar.bz2"
file_name <- "spam_2"

download.mails(url, dir_name, file_name)

[1] TRUE

## Preprocesamiento de los correos electrónicos

In [ ]:
# Hacemos una función que leea el mensaje del archivo que se le pase como parámetro
# asumimos que el archivo contiene un correo

lee_mensaje <- function(correo) {
  fd <- file(correo, open = "rt")
  lineas <- readLines(fd, warn=FALSE)
  close(fd)
  mensaje <- lineas[seq(which(lineas == "")[1] + 1, length(lineas), 1)]
  return (paste(mensaje, collapse = "\n"))
}

In [ ]:
# Creamos variables con los directorios donde se encuentran los datos
trayectoria_spam     <- file.path(dir_name, "spam_2")
trayectoria_easyham  <- file.path(dir_name, "easy_ham_2")
trayectoria_hardham  <- file.path(dir_name, "hard_ham")

### Spam

In [ ]:
# Leemos el directorio donde se encuentran los correos clasificados como spam
archivos_correos_spam <- dir(trayectoria_spam)

# quitamos el guión llamado cmds
archivos_correos_spam <- archivos_correos_spam[which(archivos_correos_spam!="cmds")] #[1:250]

archivos_correos_spam <- archivos_correos_spam[sample(1:length(archivos_correos_spam))]
archivos_correos_spam_training <- archivos_correos_spam[1:200]
archivos_correos_spam_testing <- archivos_correos_spam[201:250]
#archivos_correos_spam_testing <- archivos_correos_spam[1001:length(archivos_correos_spam)]

todo_spam <- sapply(archivos_correos_spam_training,
                   function(p) lee_mensaje(file.path(trayectoria_spam, p)))
                    
todo_spam <- enc2utf8(todo_spam)

### Easy ham

In [ ]:
# Leemos el directorio donde se encuentran los correos clasificados como ham fácilmente identificables
archivos_correos_easy_ham <- dir(trayectoria_easyham)

# quitamos el guión llamado cmds
archivos_correos_easy_ham <- archivos_correos_easy_ham[which(archivos_correos_easy_ham!="cmds")] #[1:250]

archivos_correos_easy_ham <- archivos_correos_easy_ham[sample(1:length(archivos_correos_easy_ham))]
archivos_correos_easy_ham_training <- archivos_correos_easy_ham[1:200]
archivos_correos_easy_ham_testing <- archivos_correos_easy_ham[201:250]
#archivos_correos_easy_ham_testing <- archivos_correos_easy_ham[1001:length(archivos_correos_easy_ham)]

todo_easy_ham <- sapply(archivos_correos_easy_ham_training,
                    function(p) lee_mensaje(file.path(trayectoria_easyham, p)))

todo_easy_ham <- enc2utf8(todo_easy_ham)

### Hard ham

In [ ]:
# Leemos el directorio donde se encuentran los correos clasificados como ham fácilmente identificables
archivos_correos_hard_ham <- dir(trayectoria_hardham)

# quitamos el guión llamado cmds
archivos_correos_hard_ham <- archivos_correos_hard_ham[which(archivos_correos_hard_ham!="cmds")] #[1:250]

archivos_correos_hard_ham <- archivos_correos_hard_ham[sample(1:length(archivos_correos_hard_ham))]
archivos_correos_hard_ham_training <- archivos_correos_hard_ham[1:200]
archivos_correos_hard_ham_testing <- archivos_correos_hard_ham[201:length(archivos_correos_hard_ham)]

todo_hard_ham <- sapply(archivos_correos_hard_ham_training,
                    function(p) lee_mensaje(file.path(trayectoria_hardham, p)))

todo_hard_ham <- enc2utf8(todo_hard_ham)

## Preparación de corpus y bolsa de palabras

In [ ]:
obtiene_TermDocumentMatrix <- function (vector_correos) {
  control <- list(stopwords = TRUE,
                removePunctuation = TRUE,
                removeNumbers = TRUE,
                minDocFreq = 3)
  corpus <- Corpus(VectorSource(vector_correos))
  return(TermDocumentMatrix(corpus, control))
}

### Spam

In [ ]:
spam_TDM <- obtiene_TermDocumentMatrix(todo_spam)

# Crea un data frame que provee el conjunto de caracteristicas de los datos de entrenamiento SPAM
matriz_spam <- as.matrix(spam_TDM)

conteos_spam <- rowSums(matriz_spam)
df_spam <- data.frame(cbind(names(conteos_spam),
                            as.numeric(conteos_spam)),
                      stringsAsFactors = FALSE)
names(df_spam) <- c("terminos", "frecuencia")
df_spam$frecuencia <- as.numeric(df_spam$frecuencia)
ocurrencias_spam <- sapply(1:nrow(matriz_spam),
                          function(i) # Obtiene la proporcion de documentos que contiene cada palabra
                          {
                            length(which(matriz_spam[i, ] > 0)) / ncol(matriz_spam)
                          })
densidad_spam <- df_spam$frecuencia/sum(df_spam$frecuencia,na.rm = TRUE)

df_spam <- transform(df_spam,
                     densidad = densidad_spam,
                     ocurrencias = ocurrencias_spam)

In [ ]:
head(df_spam)

terminos,frecuencia,densidad,ocurrencias
absolute,6,9.564802e-05,0.020
absolutely,27,4.304161e-04,0.080
accident,1,1.594134e-05,0.005
accidentnbsp,1,1.594134e-05,0.005
account,23,3.666507e-04,0.065
achieved,5,7.970668e-05,0.020


### Easy ham

In [ ]:
easy_ham_TDM <- obtiene_TermDocumentMatrix(todo_easy_ham)

# Crea un data frame que provee el conjunto de caracteristicas de los datos de entrenamiento easy ham
matriz_easy_ham <- as.matrix(easy_ham_TDM)

conteos_easy_ham <- rowSums(matriz_easy_ham)
df_easy_ham <- data.frame(cbind(names(conteos_easy_ham),
                            as.numeric(conteos_easy_ham)),
                      stringsAsFactors = FALSE)
names(df_easy_ham) <- c("terminos", "frecuencia")
df_easy_ham$frecuencia <- as.numeric(df_easy_ham$frecuencia)
ocurrencias_easy_ham <- sapply(1:nrow(matriz_easy_ham),
                           function(i) # Obtiene la proporcion de documentos que contiene cada palabra
                           {
                             length(which(matriz_easy_ham[i, ] > 0)) / ncol(matriz_easy_ham)
                           })
densidad_easy_ham <- df_easy_ham$frecuencia/sum(df_easy_ham$frecuencia,na.rm = TRUE)

df_easy_ham <- transform(df_easy_ham,
                     densidad = densidad_easy_ham,
                     ocurrencias = ocurrencias_easy_ham)

### Hard ham

In [ ]:
hard_ham_TDM <- obtiene_TermDocumentMatrix(todo_hard_ham)

# Crea un data frame que provee el conjunto de caracteristicas de los datos de entrenamiento easy ham
matriz_hard_ham <- as.matrix(hard_ham_TDM)

conteos_hard_ham <- rowSums(matriz_hard_ham)
df_hard_ham <- data.frame(cbind(names(conteos_hard_ham),
                            as.numeric(conteos_hard_ham)),
                      stringsAsFactors = FALSE)
names(df_hard_ham) <- c("terminos", "frecuencia")
df_hard_ham$frecuencia <- as.numeric(df_hard_ham$frecuencia)
ocurrencias_hard_ham <- sapply(1:nrow(matriz_hard_ham),
                           function(i) # Obtiene la proporcion de documentos que contiene cada palabra
                           {
                             length(which(matriz_hard_ham[i, ] > 0)) / ncol(matriz_hard_ham)
                           })
densidad_hard_ham <- df_hard_ham$frecuencia/sum(df_hard_ham$frecuencia,na.rm = TRUE)

df_hard_ham <- transform(df_hard_ham,
                     densidad = densidad_hard_ham,
                     ocurrencias = ocurrencias_hard_ham)

## Cálculo de probabilidad a posteriori

In [ ]:
a_posteriori <- function(trayectoria, df_entrenamiento, a_priori = 0.5, c = 1e-6)
{
  mensaje <- lee_mensaje(trayectoria)
  mensaje <- enc2utf8(mensaje)
  mensaje_TDM <- obtiene_TermDocumentMatrix(mensaje)
  conteos_mensaje <- rowSums(as.matrix(mensaje_TDM))

  # Encuentra palabras en data frame de entrenamiento
  mensaje_palabras_comunes <- intersect(names(conteos_mensaje), df_entrenamiento$terminos)
  
  # Ahora sólo aplicamos la clasificación Bayes ingenuo
  if(length(mensaje_palabras_comunes) < 1)
  {
    #return(a_priori * c ^ (length(conteos_mensaje)))
    return(log(a_priori) + (length(conteos_mensaje)) *log(c))
  }
  else
  {
    probabilidades_palabras_comunes <- df_entrenamiento$densidad[match(mensaje_palabras_comunes, df_entrenamiento$terminos)]
    #return(a_priori * prod(probabilidades_palabras_comunes) * c ^ (length(conteos_mensaje) - length(mensaje_palabras_comunes)))
    return(log(a_priori) + sum(log(probabilidades_palabras_comunes)) + log(c) * (length(conteos_mensaje) - length(mensaje_palabras_comunes)))
  }
}

## Clasificación

In [ ]:
clasifica_spam <- function(trayectoria, archivos) {

  hard_ham_spam_prueba <- sapply(archivos,
                             function(p) a_posteriori(file.path(trayectoria, p), df_entrenamiento = df_spam))
  hard_ham_ham_prueba <- sapply(archivos,
                            function(p) a_posteriori(file.path(trayectoria, p), df_entrenamiento = df_easy_ham))
  hard_ham_hard_ham_prueba <- sapply(archivos,
                            function(p) a_posteriori(file.path(trayectoria, p), df_entrenamiento = df_hard_ham))
  
  x <- matrix(c(ham_prueba, hard_ham_prueba, spam_prueba), nrow=length(ham_prueba), ncol=3)
  return(as.factor(argmax(x)))
}

### Hard ham

In [ ]:
# Leemos el directorio donde se encuentran los correos clasificados como ham dificlmente identificables
archivos_correos_hard_ham <- dir(trayectoria_hardham)

# quitamos el guión llamado cmds
archivos_correos_hard_ham <- archivos_correos_hard_ham[which(archivos_correos_hard_ham!="cmds")]

In [ ]:
hard_ham_res <- clasifica_spam(trayectoria_hardham, archivos_correos_hard_ham_testing)
easy_ham_res <- clasifica_spam(trayectoria_easyham, archivos_correos_easy_ham_testing)
spam_res     <- clasifica_spam(trayectoria_spam,    archivos_correos_spam_testing)

## Resultados

In [ ]:
summary(easy_ham_res)
summary(spam_res)
summary(hard_ham_res)

   Mode   FALSE    TRUE 
logical      36      14 

   Mode   FALSE    TRUE 
logical       1      49 

   Mode   FALSE    TRUE 
logical      44       6 

## metricas

In [ ]:
true_labels_hard_ham <- rep(TRUE, length(hard_ham_res))  
true_labels_easy_ham <- rep(TRUE, length(easy_ham_res)) 
true_labels_spam     <- rep(TRUE, length(spam_res))    

predicciones <- c(hard_ham_res, easy_ham_res, spam_res)
etiquetas_verdaderas <- c(true_labels_hard_ham, true_labels_easy_ham, true_labels_spam)

matriz_confusion <- table(Predicho = predicciones, Verdadero = etiquetas_verdaderas)
matriz_confusion


calcula_metricas <- function(matriz_confusion) {
  
  TP <- matriz_confusion["TRUE", "TRUE"]    
  TN <- matriz_confusion["FALSE", "FALSE"] 
  FP <- matriz_confusion["TRUE", "FALSE"]  
  FN <- matriz_confusion["FALSE", "TRUE"] 
  
  accuracy <- (TP + TN) / sum(matriz_confusion)
  
  precision <- TP / (TP + FP)
  
  recall <- TP / (TP + FN)

  return(list(
    Accuracy = accuracy,
    Precision = precision,
    Recall = recall,
  ))
}

resultados_metricas <- calcula_metricas(matriz_confusion)

resultados_metricas
